In [1]:
import pandas as pd 

#faz a leitura do arquivo csv
rock_samples = pd.read_csv('data/rocksamples.csv') 


# verificar se tudo foi carregado corretamente
* imprima as cinco primeiras linhas do novo DataFrame 

In [3]:

rock_samples.head()

,ID,Mission,Type,Subtype,Weight (g),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,125.8,88.36
1,10002,Apollo11,Soil,Unsieved,5629.0,93.73
2,10003,Apollo11,Basalt,Ilmenite,213.0,65.56
3,10004,Apollo11,Core,Unsieved,44.8,71.76
4,10005,Apollo11,Core,Unsieved,53.4,40.31


In [4]:
rock_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            2229 non-null   int64  
 1   Mission       2229 non-null   object 
 2   Type          2229 non-null   object 
 3   Subtype       2226 non-null   object 
 4   Weight (g)    2229 non-null   float64
 5   Pristine (%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB


# precisamos converter os pesos das amostras de rochas originais de gramas em quilogramas para facilitar a análise posterior dos dados. Exemplo 125,8 x 0,0001 = 0,1258kg

In [5]:
rock_samples['Weight (g)'] = rock_samples['Weight (g)'].apply(lambda x : x * 0.001)
rock_samples.rename(columns={'Weight (g)':'Weight (kg)'}, inplace=True)
rock_samples.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56
3,10004,Apollo11,Core,Unsieved,0.0448,71.76
4,10005,Apollo11,Core,Unsieved,0.0534,40.31


# Crie um DataFrame chamado *missions*, que será um resumo dos dados referentes a cada uma das seis missões Apollo que trouxeram amostras. 
# Nesse DataFrame, crie uma coluna chamada *Missão* que tenha uma linha para cada missão.

In [6]:
missions = pd.DataFrame();
missions['Mission'] = rock_samples['Mission'].unique()
missions.head()

,Mission
0,Apollo11
1,Apollo12
2,Apollo14
3,Apollo15
4,Apollo16


In [7]:
missions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Mission  6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


# soma de todas as amostras coletadas nessa missão.

In [10]:
sample_total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum()
missions = pd.merge(missions, sample_total_weight, on='Mission')
missions.rename(columns={'Weight (kg)':'Peso da Amostra (kg)'}, inplace=True)
missions

,Mission,Sample weight (kg),Peso da Amostra (kg)
0,Apollo11,21.55424,21.55424
1,Apollo12,34.34238,34.34238
2,Apollo14,41.83363,41.83363
3,Apollo15,75.39910,75.39910
4,Apollo16,92.46262,92.46262
5,Apollo17,109.44402,109.44402


# Exemplo lista como índice o valor exclusivo da coluna Missão em vez de um número:

In [11]:
sample_total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum()
sample_total_weight

Mission
Apollo11     21.55424
Apollo12     34.34238
Apollo14     41.83363
Apollo15     75.39910
Apollo16     92.46262
Apollo17    109.44402
Name: Weight (kg), dtype: float64

# adicionando mais uma coluna ao DataFrame *missions* que simplesmente captura a diferença entre a linha atual e a linha que a precede: diferença dos pesos entre as missões

In [12]:
missions['Diferença de peso'] = missions['Peso da Amostra (kg)'].diff()
missions

,Mission,Sample weight (kg),Peso da Amostra (kg),Diferença de peso
0,Apollo11,21.55424,21.55424,NaN
1,Apollo12,34.34238,34.34238,12.78814
2,Apollo14,41.83363,41.83363,7.49125
3,Apollo15,75.39910,75.39910,33.56547
4,Apollo16,92.46262,92.46262,17.06352
5,Apollo17,109.44402,109.44402,16.98140


# Preenchendo valores nulos 

In [13]:
missions['Diferença de peso'] = missions['Diferença de peso'].fillna(value=0)
missions

,Mission,Sample weight (kg),Peso da Amostra (kg),Diferença de peso
0,Apollo11,21.55424,21.55424,0.00000
1,Apollo12,34.34238,34.34238,12.78814
2,Apollo14,41.83363,41.83363,7.49125
3,Apollo15,75.39910,75.39910,33.56547
4,Apollo16,92.46262,92.46262,17.06352
5,Apollo17,109.44402,109.44402,16.98140


# crie mais seis colunas, três para os módulos lunares e três para os módulos de comando:

Nome do módulo
Massa do módulo
Diferença de massa do módulo

In [17]:
missions['Lunar module (LM)'] = ['Eagle (LM-5)', 'Intrepid (LM-6)', 'Antares (LM-8)', 'Falcon (LM-10)', 'Orion (LM-11)', 'Challenger (LM-12)']
missions['LM mass (kg)'] = [15103, 15235, 15264, 16430, 16445, 16456]
missions['LM mass diff'] = missions['LM mass (kg)'].diff()
missions['LM mass diff'] = missions['LM mass diff'].fillna(value=0)

missions['Command module (CM)'] = ['Columbia (CSM-107)', 'Yankee Clipper (CM-108)', 'Kitty Hawk (CM-110)', 'Endeavor (CM-112)', 'Casper (CM-113)', 'America (CM-114)']
missions['CM mass (kg)'] = [5560, 5609, 5758, 5875, 5840, 5960]
missions['CM mass diff'] = missions['CM mass (kg)'].diff()
missions['CM mass diff'] = missions['CM mass diff'].fillna(value=0)

missions

,Mission,Sample weight (kg),Peso da Amostra (kg),Diferença de peso,Módulo Lunar (LM),Massa do LM (kg),Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff
0,Apollo11,21.55424,21.55424,0.00000,Eagle (LM-5),15103,Eagle (LM-5),15103,0.0,Columbia (CSM-107),5560,0.0
1,Apollo12,34.34238,34.34238,12.78814,Intrepid (LM-6),15235,Intrepid (LM-6),15235,132.0,Yankee Clipper (CM-108),5609,49.0
2,Apollo14,41.83363,41.83363,7.49125,Antares (LM-8),15264,Antares (LM-8),15264,29.0,Kitty Hawk (CM-110),5758,149.0
3,Apollo15,75.39910,75.39910,33.56547,Falcon (LM-10),16430,Falcon (LM-10),16430,1166.0,Endeavor (CM-112),5875,117.0
4,Apollo16,92.46262,92.46262,17.06352,Orion (LM-11),16445,Orion (LM-11),16445,15.0,Casper (CM-113),5840,-35.0
5,Apollo17,109.44402,109.44402,16.98140,Challenger (LM-12),16456,Challenger (LM-12),16456,11.0,America (CM-114),5960,120.0


# Podemos somar alguns dos valores totais para cada missão nos módulos lunar e de comando:

In [19]:
missions['Peso Total(kg)']= missions['LM mass (kg)']+missions['CM mass (kg)']
missions['Diferença de Peso Total(kg)'] = missions['LM mass diff']+missions['CM mass (kg)']
missions

,Mission,Sample weight (kg),Peso da Amostra (kg),Diferença de peso,Módulo Lunar (LM),Massa do LM (kg),Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff,Peso Total(kg),Diferença de Peso Total(kg)
0,Apollo11,21.55424,21.55424,0.00000,Eagle (LM-5),15103,Eagle (LM-5),15103,0.0,Columbia (CSM-107),5560,0.0,20663,5560.0
1,Apollo12,34.34238,34.34238,12.78814,Intrepid (LM-6),15235,Intrepid (LM-6),15235,132.0,Yankee Clipper (CM-108),5609,49.0,20844,5741.0
2,Apollo14,41.83363,41.83363,7.49125,Antares (LM-8),15264,Antares (LM-8),15264,29.0,Kitty Hawk (CM-110),5758,149.0,21022,5787.0
3,Apollo15,75.39910,75.39910,33.56547,Falcon (LM-10),16430,Falcon (LM-10),16430,1166.0,Endeavor (CM-112),5875,117.0,22305,7041.0
4,Apollo16,92.46262,92.46262,17.06352,Orion (LM-11),16445,Orion (LM-11),16445,15.0,Casper (CM-113),5840,-35.0,22285,5855.0
5,Apollo17,109.44402,109.44402,16.98140,Challenger (LM-12),16456,Challenger (LM-12),16456,11.0,America (CM-114),5960,120.0,22416,5971.0


# Relação amostra-peso

In [29]:
saturnVPayload = 43500
missions['Crewed area : Payload'] = missions['Peso Total(kg)'] / saturnVPayload
missions['Sample : Crewed area'] = missions['Peso da Amostra (kg)'] / missions['Peso Total(kg)']
missions['Sample : Payload'] = missions['Peso da Amostra (kg)'] / saturnVPayload
missions


,Mission,Sample weight (kg),Peso da Amostra (kg),Diferença de peso,Módulo Lunar (LM),Massa do LM (kg),Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff,Peso Total(kg),Diferença de Peso Total(kg),Crewed area : Payload,Sample : Crewed area,Sample : Payload
0,Apollo11,21.55424,21.55424,0.00000,Eagle (LM-5),15103,Eagle (LM-5),15103,0.0,Columbia (CSM-107),5560,0.0,20663,5560.0,0.475011,0.001043,0.000495
1,Apollo12,34.34238,34.34238,12.78814,Intrepid (LM-6),15235,Intrepid (LM-6),15235,132.0,Yankee Clipper (CM-108),5609,49.0,20844,5741.0,0.479172,0.001648,0.000789
2,Apollo14,41.83363,41.83363,7.49125,Antares (LM-8),15264,Antares (LM-8),15264,29.0,Kitty Hawk (CM-110),5758,149.0,21022,5787.0,0.483264,0.001990,0.000962
3,Apollo15,75.39910,75.39910,33.56547,Falcon (LM-10),16430,Falcon (LM-10),16430,1166.0,Endeavor (CM-112),5875,117.0,22305,7041.0,0.512759,0.003380,0.001733
4,Apollo16,92.46262,92.46262,17.06352,Orion (LM-11),16445,Orion (LM-11),16445,15.0,Casper (CM-113),5840,-35.0,22285,5855.0,0.512299,0.004149,0.002126
5,Apollo17,109.44402,109.44402,16.98140,Challenger (LM-12),16456,Challenger (LM-12),16456,11.0,America (CM-114),5960,120.0,22416,5971.0,0.515310,0.004882,0.002516


# Podemos usar a função *mean() para calcular a média* de todas essas proporções em todas as missões.
*crewedArea* => Area Tripulada
*Sample* =>Amostra
*Payload* => Carga Útil
*Crewed Area/Payload Ratio* => (Razão Área Tripulada/Carga Útil): É uma medida que indica a proporção entre o espaço disponível para a tripulação e a carga útil transportada pela espaçonave. Quanto maior essa razão, mais espaço relativo é dedicado à tripulação em comparação com a carga útil.
*sample_crewedArea_ratio* parece indicar a relação entre a área ocupada pelas amostras coletadas em uma espaçonave e a área total disponível para a tripulação.
*sample_payload_ratio* parece indicar a relação entre o peso das amostras coletadas em uma espaçonave e o peso total da carga útil (payload) transportada por essa espaçonave

In [30]:
crewedArea_payload_ratio = missions['Crewed area : Payload'].mean()
sample_crewedArea_ratio = missions['Sample : Crewed area'].mean()
sample_payload_ratio = missions['Sample : Payload'].mean()
print(crewedArea_payload_ratio)
print(sample_crewedArea_ratio)
print(sample_payload_ratio)

0.4963026819923371
0.002848764392685611
0.0014369195019157087


# criar um DataFrame com as informações que temos sobre as três missões tripuladas

In [32]:
artemis_crewedArea = 26520
artemis_mission = pd.DataFrame({'Missão':['artemis1','artemis1b','artemis2'],
                                'Peso Total (kg)':[artemis_crewedArea,artemis_crewedArea,artemis_crewedArea],
                                 'Carga Útil(kg)':[26988, 37965, 42955]})
artemis_mission



,Missão,Peso Total (kg),Carga Útil(kg)
0,artemis1,26520,26988
1,artemis1b,26520,37965
2,artemis2,26520,42955


# estimar o peso das amostras com base nas proporções que determinamos usando as missões da Apollo

In [36]:
artemis_mission['Peso da Amostra total (kg)'] = artemis_mission['Peso Total (kg)'] * sample_crewedArea_ratio
artemis_mission['Peso da Amostra da Carga Útil (kg)'] = artemis_mission['Carga Útil(kg)'] * sample_payload_ratio
artemis_mission

,Missão,Peso Total (kg),Carga Útil(kg),Peso da Amostra from total (kg),Peso da Amostra from payload (kg),Peso da Amostra from Carga Útil (kg),Peso da Amostra total (kg),Peso da Amostra da Carga Útil (kg)
0,artemis1,26520,26988,75.549232,38.779584,38.779584,75.549232,38.779584
1,artemis1b,26520,37965,75.549232,54.552649,54.552649,75.549232,54.552649
2,artemis2,26520,42955,75.549232,61.722877,61.722877,75.549232,61.722877


# obter a média das duas previsões

In [38]:
artemis_mission['Estimativa do Peso da Amostra (kg)'] = (artemis_mission['Peso da Amostra da Carga Útil (kg)'] + artemis_mission['Peso da Amostra total (kg)'])/2
artemis_mission

,Missão,Peso Total (kg),Carga Útil(kg),Peso da Amostra from total (kg),Peso da Amostra from payload (kg),Peso da Amostra from Carga Útil (kg),Peso da Amostra total (kg),Peso da Amostra da Carga Útil (kg),Estimativa do Peso da Amostra (kg)
0,artemis1,26520,26988,75.549232,38.779584,38.779584,75.549232,38.779584,57.164408
1,artemis1b,26520,37965,75.549232,54.552649,54.552649,75.549232,54.552649,65.050940
2,artemis2,26520,42955,75.549232,61.722877,61.722877,75.549232,61.722877,68.636054


# aprender a limpar e manipular os dados.
# o quanto resta de cada amostra trazida das missões Apollo, multiplicando o peso da amostra coletada originalmente pelo percentual da amostra íntegra restante.
*Remaining (kg)* =>Restante
*Pristine (%)* =>Integridade


In [42]:
rock_samples['Remaining (kg)'] = rock_samples['Weight (kg)'] * (rock_samples['Pristine (%)'] * .01)
rock_samples.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%),Remaining (kg)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36,0.111157
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73,5.276062
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56,0.139643
3,10004,Apollo11,Core,Unsieved,0.0448,71.76,0.032148
4,10005,Apollo11,Core,Unsieved,0.0534,40.31,0.021526


# DataFrame rock_samples não é útil para examinar mais de 2.000 amostras. Para entender melhor o que o conjunto de dados contém, você pode usar a função describe():

In [43]:
rock_samples.describe()

,ID,Weight (kg),Pristine (%),Remaining (kg)
count,2229.000000,2229.000000,2229.000000,2229.000000
mean,52058.432032,0.168253,84.512764,0.138103
std,26207.651471,0.637286,22.057299,0.525954
min,10001.000000,0.000000,0.000000,0.000000
25%,15437.000000,0.003000,80.010000,0.002432
50%,65527.000000,0.010200,92.300000,0.008530
75%,72142.000000,0.093490,98.140000,0.078240
max,79537.000000,11.729000,180.000000,11.169527


# extraindo apenas as amostras que estão próximas de acabar

In [44]:
amostras_baixas = rock_samples.loc[(rock_samples['Weight (kg)'] >= .16) & (rock_samples['Pristine (%)'] <= 50)]
amostras_baixas.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%),Remaining (kg)
11,10017,Apollo11,Basalt,Ilmenite,0.973,43.71,0.425298
14,10020,Apollo11,Basalt,Ilmenite,0.425,27.88,0.118490
15,10021,Apollo11,Breccia,Regolith,0.250,30.21,0.075525
29,10045,Apollo11,Basalt,Olivine,0.185,12.13,0.022441
37,10057,Apollo11,Basalt,Ilmenite,0.919,35.15,0.323028


In [45]:
amostras_baixas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              27 non-null     int64  
 1   Mission         27 non-null     object 
 2   Type            27 non-null     object 
 3   Subtype         27 non-null     object 
 4   Weight (kg)     27 non-null     float64
 5   Pristine (%)    27 non-null     float64
 6   Remaining (kg)  27 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.7+ KB


# usar a função unique() para ver quantos tipos exclusivos temos nos DataFrames amostras baixas e amostras de rocha.


In [46]:
amostras_baixas.Type.unique()

array(['Basalt', 'Breccia', 'Soil', 'Core'], dtype=object)

In [47]:
rock_samples.Type.unique()

array(['Soil', 'Basalt', 'Core', 'Breccia', 'Special', 'Crustal'],
      dtype=object)

# Calcular no DataFrame amostras baixas, quantos de cada tipo são considerados pouca quantidade?

In [48]:
amostras_baixas.groupby('Type')['Weight (kg)'].count()

Type
Basalt     14
Breccia     8
Core        1
Soil        4
Name: Weight (kg), dtype: int64

# coletando informações sobre amostras especificas

In [49]:
amostras_necessarias = amostras_baixas[amostras_baixas['Type'].isin(['Basalt', 'Breccia'])]
amostras_necessarias.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              22 non-null     int64  
 1   Mission         22 non-null     object 
 2   Type            22 non-null     object 
 3   Subtype         22 non-null     object 
 4   Weight (kg)     22 non-null     float64
 5   Pristine (%)    22 non-null     float64
 6   Remaining (kg)  22 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.4+ KB


# comparar o peso total do DataFrame amostras necessarias com o DataFrame amostras de rocha.

In [50]:
amostras_necessarias.groupby('Type')['Weight (kg)'].sum()

Type
Basalt     17.4234
Breccia    10.1185
Name: Weight (kg), dtype: float64

In [51]:
rock_samples.groupby('Type')['Weight (kg)'].sum()

Type
Basalt      93.14077
Breccia    168.88075
Core        19.93587
Crustal      4.74469
Soil        87.58981
Special      0.74410
Name: Weight (kg), dtype: float64

# adicionar as rochas Crustais ao conjunto de amostras necessárias:

In [52]:
amostras_necessarias = pd.concat([amostras_necessarias,rock_samples.loc[rock_samples['Type'] == 'Crustal']])
amostras_necessarias.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68 entries, 11 to 2189
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              68 non-null     int64  
 1   Mission         68 non-null     object 
 2   Type            68 non-null     object 
 3   Subtype         68 non-null     object 
 4   Weight (kg)     68 non-null     float64
 5   Pristine (%)    68 non-null     float64
 6   Remaining (kg)  68 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 4.2+ KB


#  uma coluna para cada tipo de rocha das quais desejamos mais amostras

In [53]:
visao_geral_amostras_necessarias = pd.DataFrame()
visao_geral_amostras_necessarias['Type'] = amostras_necessarias.Type.unique()
visao_geral_amostras_necessarias

,Type
0,Basalt
1,Breccia
2,Crustal


#  peso total de cada tipo de rocha coletado originalmente

In [55]:
peso_amostras_necessarias = amostras_necessarias.groupby('Type')['Weight (kg)'].sum().reset_index()
visao_geral_amostras_necessarias = pd.merge(visao_geral_amostras_necessarias, peso_amostras_necessarias, on='Type')
visao_geral_amostras_necessarias.rename(columns={'Weight (kg)':'Peso Total (kg)'}, inplace=True)
visao_geral_amostras_necessarias

,Type,Total weight (kg),Peso Total (kg)
0,Basalt,17.42340,17.42340
1,Breccia,10.11850,10.11850
2,Crustal,4.74469,4.74469


# informar o tamanho estimado de cada tipo de rocha, isso poderá facilitar o processo de coleta

In [57]:
peso_medio_amostras_necessarias = amostras_necessarias.groupby('Type')['Weight (kg)'].mean().reset_index()
visao_geral_amostras_necessarias = pd.merge(visao_geral_amostras_necessarias, peso_medio_amostras_necessarias, on='Type')
visao_geral_amostras_necessarias.rename(columns={'Weight (kg)':'Peso médio(kg)'}, inplace=True)
visao_geral_amostras_necessarias

,Type,Total weight (kg),Peso Total (kg),Average weight (kg),Peso médio(kg)
0,Basalt,17.42340,17.42340,1.244529,1.244529
1,Breccia,10.11850,10.11850,1.264813,1.264813
2,Crustal,4.74469,4.74469,0.103145,0.103145


# usar o número total de cada tipo e obter o percentual restante de cada tipo de rocha.

In [63]:
total_rock_count = rock_samples.groupby('Type')['ID'].count().reset_index()
visao_geral_amostras_necessarias= pd.merge(visao_geral_amostras_necessarias, total_rock_count, on='Type')
visao_geral_amostras_necessarias.rename(columns={'ID':'Number of samples'}, inplace=True)
total_rocks = visao_geral_amostras_necessarias['Number of samples'].sum()
visao_geral_amostras_necessarias['Percentage of rocks'] = visao_geral_amostras_necessarias['Number of samples'] / total_rocks
visao_geral_amostras_necessarias

,Type,Total weight (kg),Peso Total (kg),Average weight (kg),Peso médio(kg),Numero de Amostras,Percentage of rocks,Numero de Amostras,Numero de Amostras,Numero de Amostras,Numero de Amostras,Number of samples
0,Basalt,17.42340,17.42340,1.244529,1.244529,351,0.258850,351,351,351,351,351
1,Breccia,10.11850,10.11850,1.264813,1.264813,959,0.707227,959,959,959,959,959
2,Crustal,4.74469,4.74469,0.103145,0.103145,46,0.033923,46,46,46,46,46


# determinar o peso médio das amostras que estimamos na unidade anterior

In [68]:
artemis_ave_weight = artemis_mission['Estimativa do Peso da Amostra (kg)'].mean()
artemis_ave_weight 

63.61713411579792

# determinar quantas de cada rocha queremos que os astronautas busquem coletar

In [69]:
visao_geral_amostras_necessarias['Weight to collect'] = visao_geral_amostras_necessarias['Percentage of rocks'] * artemis_ave_weight
visao_geral_amostras_necessarias['Rocks to collect'] = visao_geral_amostras_necessarias['Weight to collect'] / visao_geral_amostras_necessarias['Average weight (kg)']
visao_geral_amostras_necessarias

,Type,Total weight (kg),Peso Total (kg),Average weight (kg),Peso médio(kg),Numero de Amostras,Percentage of rocks,Numero de Amostras,Numero de Amostras,Numero de Amostras,Numero de Amostras,Number of samples,Weight to collect,Rocks to collect
0,Basalt,17.42340,17.42340,1.244529,1.244529,351,0.258850,351,351,351,351,351,16.467267,13.231731
1,Breccia,10.11850,10.11850,1.264813,1.264813,959,0.707227,959,959,959,959,959,44.991764,35.571884
2,Crustal,4.74469,4.74469,0.103145,0.103145,46,0.033923,46,46,46,46,46,2.158103,20.922917
